# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
DF=pd.read_csv('WeatherData.csv',index_col=0)
DF['Humidity']=DF['Humidity'].astype(float)
DF

,City,Country,Lat,Lng,Cloudiness,Humidity,High Temp,Wind Speed
0,George Town,MY,5.41,100.34,20,94.0,78.80,5.82
1,Whitehorse,CA,60.72,-135.05,90,92.0,26.60,8.05
2,Ossora,RU,59.24,163.07,100,66.0,2.26,20.36
3,Hong Kong,HK,22.29,114.16,40,68.0,64.99,8.05
4,Pangai,TO,-19.80,-174.35,100,82.0,80.91,18.21
...,...,...,...,...,...,...,...,...
572,Tibati,CM,6.47,12.63,0,12.0,95.40,12.44
573,Santiago del Estero,AR,-27.80,-64.26,100,79.0,65.50,3.40
574,Jiayuguan,CN,39.82,98.30,0,19.0,33.26,8.21
575,Batagay-Alyta,RU,67.80,130.41,18,96.0,-29.81,3.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locs=DF[['Lat','Lng']]
weight=DF['Humidity']


In [4]:
fig=gmaps.figure()

heat=gmaps.heatmap_layer(locs,weights=weight,max_intensity=DF['Humidity'].max())

fig.add_layer(heat)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df=DF[(DF['High Temp']>70)&(DF['High Temp']<80)&(DF['Cloudiness']==0)&(DF['Wind Speed']<10)].dropna()
hotel_df.reset_index(inplace=True,drop=True)
hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,High Temp,Wind Speed
0,Port Alfred,ZA,-33.59,26.89,0,83.0,71.01,3.00
1,San Vicente,AR,-26.62,-54.13,0,99.0,75.99,5.03
2,Port Augusta,AU,-32.50,137.77,0,78.0,73.40,6.93
3,Lebu,CL,-37.62,-73.65,0,47.0,73.47,9.42
4,Sihor,IN,21.70,71.97,0,60.0,73.40,5.82
5,Tautira,PF,-17.73,-149.15,0,83.0,77.00,2.24
6,Port Lincoln,AU,-34.73,135.87,0,72.0,70.77,6.80
7,Rafaela,AR,-31.25,-61.49,0,39.0,77.00,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels=[]
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "resort",
    "key": g_key
    }

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] = f"{lat},{lng}"

    resp=requests.get(url, params=params).json()
    try:
        hotel=resp['results'][0]['name']
        hotels.append(hotel)
        print(f'{hotel} in {row["City"]}.')
    except:
        hotels.append('No Hotels')
        print(f'No Hotel close to {row["City"]}.')

hotel_df['Hotel']=hotels


Green Fountain Farm Caravan Park in Port Alfred.
Complejo DER WALD in San Vicente.
Arkaroola Wilderness Sanctuary in Port Augusta.
Centro Turistico Los Chilcos in Lebu.
Shubhdev Revival Resort in Sihor.
InterContinental Resort Tahiti in Tautira.
Port Lincoln Cabin Park - Accommodation Caravan Park in Port Lincoln.
Hotel Campo Alegre in Rafaela.


In [7]:
hotel_df

,City,Country,Lat,Lng,Cloudiness,Humidity,High Temp,Wind Speed,Hotel
0,Port Alfred,ZA,-33.59,26.89,0,83.0,71.01,3.00,Green Fountain Farm Caravan Park
1,San Vicente,AR,-26.62,-54.13,0,99.0,75.99,5.03,Complejo DER WALD
2,Port Augusta,AU,-32.50,137.77,0,78.0,73.40,6.93,Arkaroola Wilderness Sanctuary
3,Lebu,CL,-37.62,-73.65,0,47.0,73.47,9.42,Centro Turistico Los Chilcos
4,Sihor,IN,21.70,71.97,0,60.0,73.40,5.82,Shubhdev Revival Resort
5,Tautira,PF,-17.73,-149.15,0,83.0,77.00,2.24,InterContinental Resort Tahiti
6,Port Lincoln,AU,-34.73,135.87,0,72.0,70.77,6.80,Port Lincoln Cabin Park - Accommodation Carava...
7,Rafaela,AR,-31.25,-61.49,0,39.0,77.00,4.00,Hotel Campo Alegre


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markerlayer=gmaps.marker_layer(hotel_df[['Lat','Lng']],info_box_content=hotel_info)
fig.add_layer(markerlayer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))